# Tutorial Model Generation

**Pyrit’s** modeling and simulation workflow is organized in the following models: *problem definition, problem solution and post-processing*. In the first step, i.e., *problem definition*, the geometry is defined using the open source software *Gmsh*. 
**Pyrit** offers several ways ways to generate a model: The user can [import a step file](#Approach-1:-Model-definition-based-on-STEP-file), create a geometry [using **Pyrit's** *geometry* package](#Approach-2:-Model-definition-based-on-Pyrit), or [import an already existing *Gmsh* file](#Approach-3:-Model-definition-based-on-geo-file) (`*.geo`-file). 

The "best" way always depends on the already available files and the complexity of the model. This tutorial shows the three different model generation approaches. First, we show how to generate a model based on an existing STEP file (`*.stp`-file). Second, the model is created based on an existing geo file, and, thirdly, we generate the geometry using the Pyrit *geometry* package.

## Approach 1: Model definition based on STEP file

### A) Import the geometry data to Gmsh

Often, the geometry and topology information, respectively, of a simulation problem  already exists, e.g. in a different simulation software like *CST* or *COMSOL* or in a CAD file format. Most software environments offer the export of the geometry data in the STEP file format. This section shows how to import and create a geometry in **Pyrit** data based on a STEP file.

Note that e.g. in CST, a STEP file can be generated via ``Export ->3D Files->3D General ->STEP file``.

First, we check the geometry visually in *Gmsh* and adapt the geometry. The following command opens the STEP file in GMSH (before: set GMSH as program to open STEP files.)


In [ ]:
from pyrit.geometry import Geometry
import gmsh
geo = Geometry("",show_gui=True)
with geo:
    gmsh.merge('Tutorial_STEP.stp')

With a look at the list of the shapes (``Tools->Visibility or [Strg+Shift+V]``), we notice that the faces are not connected and the curves and the points are duplicated. This needs to be repaired, before the model can be simulated. Therefore, we create a valid *Gmsh* file. We open the `Tutorial_STEP.geo`-file in a text editor (e.g. notepad++ or Pycharm) and insert:
```C++
SetFactory("OpenCASCADE");
v() = ShapeFromFile("ModelName.stp");
BooleanFragments{ Surface{v()}; Delete; }{}
```
Here, ModelName denotes the name of the STEP file, i.e. ``Tutorial_STEP.stp``. In the next step, save this file as  ".geo".
Now, all the duplicated curves and points are removed. In most cases this is sufficient for creating a valid *Gmsh* file. However, in some cases two edges may not be exactly identical. Then, this procedure fails, and some edges need to be deleted manually. To check, if an edge is correctly defined, we hover over the edge in the *Gmsh* GUI. The boundary should be allocated to two surfaces like:\
"On boundary of surfaces: x, y"\
Here, *x* and *y* denote the corresponding numbers of the surfaces. If this is not the case, the model can be adjusted with:
```C++
Delete {Surface{SurfaceNumber};}
Delete {Line{LineNumber};}
Delete {Point{PointNumber};}
Line Loop(LoopNumber) = {LineNumber};
Plane Surface(SurfaceNumber) = {LoopNumber};
```
Here `SurfaceNumber, LineNumber, PointNumber` and `LoopNumber` denote the number of the corresponding shape. In the case that more than one shape needs to be deleted, this can be done by `{1,2,3,4}` or `{1:4}`. Note that an edge cannot be deleted, if it is still connected to a surface. Therefore, in order to delete an edge, first the surface needs to be deleted. Same holds for points and edges. 

In order to create a new surface, first a `Line Loop` has to be created, which contains all edges belonging to the new surface. These edges need to be ordered. It does not matter whether the order is clockwise or counterclockwise. Finally the new surface is defined via the `Line Loop`.
__________________________________________

Once the geometry is defined correctly, the `Physical Surfaces` for the definition of the material properties and the `Physical Line` for the definition of the boundary conditions are created:
```C++
    Physical Surface('MaterialName') = {SurfaceNumber};
    Physical Line('LineName') = {LineNumber}
```  
In our example: 
```C++
    Physical Surface('rotor') = {1};
    Physical Surface('steel') = {2};
    Physical Surface('stator') = {3};
    Physical Line('external_temperature') = {5,6};
```
to the file. In this example, the surface belonging to the SurfaceNumber {1} gets added to the physical surface *rotor*. The lines with the LineNumber {5,6} the boundaries of the computational domain, where the external temperature is defined.\
Note: The numbering of the surface and the line can be checked in the *Gmsh* GUI with (`Tools->Visibility or [Strg+Shift+V]`).

In [ ]:
# Our valid Gmsh file should now look like:
import pandas as pd
code_in_geofile=pd.read_fwf('Tutorial_STEP.geo') 
print(code_in_geofile)

_________________________________________________

An **alternative way** to generate a valid *Gmsh* file is to open the STEP file in teh *Gmsh* GUI and create a new `*.geo`-file via `Modules->Geometry->Elementary entities->Set geometry kernel-> OpenCASCADE\`
This creates:
```C++
    Merge "Tutorial_STEP.stp";
    //+
    SetFactory("OpenCASCADE");
```
Here, the duplicated curves and points are not deleted.\
This is done as explained above with:
```C++
    Delete {Surface{1};}
    Delete {Line{1:4};}
    Delete {Point{1:4};}
    //careful with the number of the Loop. It is not allowed to have curves with the same number
    Line Loop(15) = {5,6}; 
    Line Loop(16) = {7,8};
    Plane Surface(4) = {15,16};
```
Again, the `Physical Surface` and `Physical Line` are defined by:
```C++
    Physical Surface('rotor') = {4};
    Physical Surface('steel') = {2};
    Physical Surface('stator') = {3};
    Physical Line('external_temperature') = {9,10};
``` 

Note: One can see that the numeration of the surface and the line has changed in comparison to the other example. However, both files represent the same model.

In [ ]:
# This shows the code of another valid Gmsh file:
import pandas as pd
code_in_geofile=pd.read_fwf('Tutorial_STEP_2.geo') 
print(code_in_geofile)

### B) Problem definition workflow

After generating a valid *Gmsh* file, the geometry can be imported to **Pyrit**. Subsequently, we can model the simulation problem. In this tutorial example, a transient thermal problem is created, and a simple geometry of three concentric cylinders is considered. These three cylinder represent a simplified geometry of a motor with the domains "steel" (i.e. the shaft), "rotor" and "stator". The start temperature is 273.15K and the surrounding temperature will rise over time to 293.15K.

The **Pyrit**-Python-file starts with the imports.

In [ ]:
import gmsh
import numpy as np
from matplotlib import pyplot as plt
from dataclasses import dataclass

from pyrit.excitation import Excitations
from pyrit.material import Mat, Materials, ThermalConductivity, VolumetricHeatCapacity
from pyrit.problem import ThermalProblemCartTransient

from pyrit.region import Regions
from pyrit.shapefunction import TriCartesianNodalShapeFunction
from pyrit.bdrycond.BdryCond import BdryCond, BCDirichlet
from pyrit.geometry.Geometry import Geometry
from pyrit.mesh import TriMesh
from pyrit.toolbox.ImportGmshToolbox import geo_to_msh, read_msh_file, read_physical_groups, generate_regions

Next, the simulation settings and definitions are provided in a `dataclass`.

In [ ]:
@dataclass
class Settings:
    path_dir: str = ''                   #: in case the file is stored somewhere else
    file_name: str = 'Tutorial_STEP.geo' #: name of geo-file
    outfile_name: str = 'Tutorial_STEP'  #: name of geo-file
    write_mesh: bool = True              #: Create new msh-file from geo-file
    refinement_steps: int = 0            #: number of refinement steps
    show_gui: bool = True                #: Show GMSH gui
    mesh_size: float = 0.2               #: Meshsize (default=1)
    kelvin: bool = False                 #: Temperature Unit (False=>degC)

Additionally, a time dependent excitation function, i.e. the temperature rise is defined. In the simulation, this corresponds to a time-varying Dirichlet-type boundary condition. In our example, the temperature rises linearly from 273.15 K to 293.15 K.

In [ ]:
class Boundaryconditions:
    """Time dependent temperature class"""

    def __init__(self, time=0):
        self.time = time

    def __call__(self, time=None):
        if time is None:
            time = self.time
        return 273.15+20*time/12000 # An increase of the temperature over time in sec (Endtime: t_end=12000 s).

In the class with the model name, all the important properties of the model are defined. These are the time steps, the materials, the boundary conditions, the mesh, the regions, the shapefunctions of the underlying finite element procedure, and the problem.

In [ ]:
class Tutorial_STEP_Model:
    def __init__(self, t_end: float = 12000, number_of_time_steps: int = 100, settings: Settings = None):
        if settings is None:
            settings = Settings()
        self.settings = settings
        self._mesh: TriMesh = None
        self._regions: Regions = None
        self._materials: Materials = None
        self._boundary_cond: BdryCond = None
        self._sft: TriCartesianNodalShapeFunction = None

        self.t_end = t_end
        self.number_of_time_steps = number_of_time_steps
        self._problem = None

        self.temperature_start = np.ones((self.mesh.num_node,)) * 273.15

    @property
    def time_steps(self):
        return np.linspace(0, self.t_end, self.number_of_time_steps)

    @property
    def materials(self):
        if self._materials is None:
            # Mat('name',ThermalConductivity(Lambda [W/(m*K)]), VolumetricHeatCapacity(Rho[Kg/m^3] * SpecificHeat[J/(kg*K)]))
            rotor = Mat('rotor', ThermalConductivity(20), VolumetricHeatCapacity(7000 * 500))
            stator = Mat('stator', ThermalConductivity(40), VolumetricHeatCapacity(7500 * 500))
            steel = Mat('steel', ThermalConductivity(60), VolumetricHeatCapacity(8000 * 480))
            self._materials = Materials(rotor, stator, steel)
        return self._materials

    @property
    def boundary_cond(self):
        if self._boundary_cond is None:
            external_temperature = BCDirichlet(Boundaryconditions(), 'external_temperature')
            self._boundary_cond = BdryCond(external_temperature)
        return self._boundary_cond

    @property
    def mesh(self):
        if self._mesh is None:
            if self.settings.write_mesh:
                msh_path = geo_to_msh(self.settings.file_name, refinement_steps=self.settings.refinement_steps,
                                      show_gui=self.settings.show_gui, mesh_size_factor=self.settings.mesh_size)
                msh = read_msh_file(msh_path, mesh_type=TriMesh)
                msh.node = msh.node / 1000  # Convert mm to m
                self._mesh = msh
            else:
                msh = read_msh_file(self.settings.file_name, mesh_type=TriMesh)
                msh.node = msh.node / 1000  # Convert mm to m
                self._mesh = msh
        return self._mesh

    @mesh.setter
    def mesh(self, msh):
        self._mesh = msh
        self._sft = None

    @property
    def regions(self):
        if self._regions is None:
            self._regions = generate_regions(self.settings.file_name, self.materials, self.boundary_cond)
        return self._regions

    @property
    def sft(self):
        if self._sft is None:
            self._sft = TriCartesianNodalShapeFunction(self.mesh)
        return self._sft

    @property
    def problem(self) -> ThermalProblemCartTransient:
        if self._problem is None:
            self._problem = ThermalProblemCartTransient('tutorial_STEP_model', self.mesh, self.sft,
                                                              self.regions, self.materials,
                                                              self.boundary_cond, Excitations(),
                                                              self.time_steps)
        return self._problem



In order to create the problem, the class is called by:

In [ ]:
#Note: This should open GMSH and show the model with its mesh.
model = Tutorial_STEP_Model()

## Approach 2: Model definition based on Pyrit

The second approach uses solely Pyrit through the *Gmsh* API. Thus, no extra files are necessary. 

We show this approach for a different model than the motor of approach 1. Now, we model a **torus shaped conductor** of copper in a **magnetostatic setting**. It is surrounded by large cylinder filled with air, i.e. the computational domain. Because of the symmetry, only one cut through the domain at one angle in cylindrical coordinates is created, i.e. we model the setup in the $\rho-z$-plane. 

The model is represented by a dataclass that handles properties of the model, e.g. the geometrical parameters and material parameters. A dedicated method of the class creates the model. Inside of this method, the geometry is built, all data structures needed for further computation are generated and materials, boundary conditions and excitations arer assigned to the geometry. Finally, all that is combined in a specific problem class. 

In the following, the code is first shown and explained in smaller parts. After that, the whole code is summarized and executed for a simple example. 

### Head of the class
The model is implemented as a Python dataclass in Python. The head of the class looks for example like in the following cell. 
Basic properties are provided firstly, i.e. the distances or the current through the conductor. The class can also be used to compute intermediate values, as for example the current density in the conductor. 

```python
@dataclass
class ConductorLoop:
    width: float = 2.
    height: float = 4.
    center_conductor_x: float = 1.
    center_conductor_y: float = 2.
    radius_conductor: float = 0.2

    current: float = 1.
    conductivity: float = 1e6
        
    @property
    def current_density(self) -> float:
        """The current density in the conductor"""
        return self.current / (np.pi * self.radius_conductor ** 2)
```

### Method create_problem
The dataclass contains at least a method like the following example to create the geometry. 
Here, with the argument ``refinement_steps``, the number of refinement steps in the meshing process is provdied. Furthermore, with ``kwargs`` key word arguments can be passed to the constructor of Geometry. This allows a high flexibility for later on. 

```python
    def create_problem(self, refinement_steps: int = 0, **kwargs) -> MagneticProblemAxiStatic:
        """Create the problem.

        Parameters
        ----------
        refinement_steps : int
            Number of refinement steps in the meshing process.
        kwargs :
            Keyword arguments passed to the constructor of ``Geometry``

        Returns
        -------
        problem : MagneticProblemAxiStatic
            A problem instance
        """
        geo = Geometry(model_name="Conductor Loop", **kwargs)
```

#### Materials
First, we define the materials, in our example air and copper. Each is an instance of ``Mat`` and holds a number of material properties. In this case, we define Reluctivity and Conductivity because this are the relevant properties for the magnetostatic problem. 

All materials are added to a ``Materials`` object, which manages all materials of a problem.

```python
        air = Mat('air', Reluctivity(nu_0), Conductivity(0))
        copper = Mat('copper', Reluctivity(nu_0), Conductivity(self.conductivity))
        materials = Materials(air, copper)
```

#### Boundary conditions
Next, the boundary conditions are defined. Here, we only need homogeneous Dirichlet boundary conditions, which correspond to a vanishing normal magnetic flux density on the boundary. 

All boundary conditions (here only one) are added to a ``BdryCond`` object. This is responsible for managing all the different boundary conditions present in a problem.

```python
        outer_bc = BCDirichlet(0) # magnetic vector potential = 0 on the boundary
        boundary_cond = BdryCond(outer_bc)
```

#### Excitations
Furthermore, we define the excitations. In our case, it is a current density within the conductor. The property is used here to get the correct current density, since in the constructor only the radius and the total current are specified. 

Analogously to materials and boundary conditions, the excitations are added to an ``Excitations`` object that manages all excitations of a problem. 

```python
        exci = CurrentDensity(self.current_density, 'cd')
        excitations = Excitations(exci)
```

#### Physical groups and assignments
For assigning materials, boundary conditions and excitations to parts of the geometry, a approach from *Gmsh* is used. The assignment does not directly use geometrical entities (lines or surfaces) but uses *physical groups*. A physical group is a collection of geometrical entities of the same dimension. Based on these, a material can be assigned easily to different areas of the geometry by collecting all these areas in one physical group. 

The physical groups are generated with the ``geo`` object, that is an instance of ``Geometry``. A unique tag and the dimension of the physical group are defined. A name is optional but advantageous. 

```python
        pg_conductor = geo.create_physical_group(tag=1, dim=2, name="conductor")
        pg_surrounding = geo.create_physical_group(tag=2, dim=2, name="surrounding")
        pg_boundary = geo.create_physical_group(tag=3, dim=1, name="boundary")
```

Now, the materials, the boundary condition, and the excitation are assigned to the physical group of the `` geo`` object

```python
        geo.add_material_to_physical_group(air, pg_surrounding)
        geo.add_material_to_physical_group(copper, pg_conductor)
        geo.add_boundary_condition_to_physical_group(outer_bc, pg_boundary)
        geo.add_excitation_to_physical_group(exci, pg_conductor)
```

#### Geometry creation
With the above definitions, we create the geometry using the ``geo`` object. It is a context manager that handles the initialization and shutdown of *Gmsh*. Inside of this context manager, all *Gmsh* commands can be used

```python 
        with geo:
            rectangle = gmsh.model.occ.addRectangle(0, 0, 0, self.width, self.height)
            disk = gmsh.model.occ.addDisk(self.center_conductor_x, self.center_conductor_y, 0, self.radius_conductor,
                                          self.radius_conductor)
            gmsh.model.occ.cut([[2, rectangle]], [[2, disk]], removeTool=False)
```

#### Assignment of entities to physical groups
Finally, we assign geometrical entities to the physical groups. Entities are represented by their tags. The tag can be seen in the GUI or via the API. In this example, the tags of the physical group representing the boundary conditions were retrieved from the GUI. It can be opened either with an option ``show_gui=True`` on the ``geo``-object or with the command ``geo.open_gui()``. 

```python
            pg_conductor.add_entity(disk)
            pg_boundary.add_entity(6, 8, 9)
            pg_surrounding.add_entity(rectangle)
```

#### Mesh generation and extraction of the mesh and regions
The description of the model is now complete. We now process the data structures required for the simulation process, i.e. the ``mesh`` and the ``regions``. 

For the mesh generation, the dimension is specified. After that, the mesh can be refined. The last step reads the mesh from *Gmsh*. Again, the dimension of the mesh has to be specified. 
The ``geo.get_mesh`` method returns in general a ``TriMesh`` (triangular mesh) or a ``TetMesh`` (tetrahedral mesh). Sonce we have a specified the dimension to 2, the returned mesh is a ``TriMesh``. 

When the problem is in axisymmetric coordinates, the ``TriMesh`` has to converted to an ``AxiMesh``. This is here the case. The conversion takes place in the last line of the following block. 

```python
            geo.create_mesh(dim=2)
            for _ in range(refinement_steps):
                geo.refine_mesh()
            mesh = geo.get_mesh(dim=2)
            mesh = AxiMesh.generate_axi_mesh(mesh)
```

The regions can be generated as shown in the next code block. The ``region`` is the discretized counterpart of the ``pyhsical group``. Discrete geometrical entities (such as triangles or edges in the mesh) are assigned to a region. The regions, in turn, contain the information of materials, boundary conditions and excitations defined on itself. 

```python
            regions = geo.get_regions()
```

#### Creation of the shape function object
With the mesh, we create the shape function object that will later create the finite element matrices. This part does not need to be in the context manager any more. 

```python
        shape_function = TriAxisymmetricEdgeShapeFunction(mesh)
```

#### Creation of the Problem
We now have a lot of different objects that contain different information about our problem. The last step of this method is to combine them in one problem-class. So all data structures are tied together and further handling is simplified. 

In the next step, we initialize the magnetostatic problem in axisymmetric coordinates with the template-class ``MagneticProblemAxiStatic``. The constructor expects all of the created data structures. The problem object itself performs consistency and type checks to prevent faulty inputs. It also provides a ``solve``-method to solve the problem in a standardized way. Finally, the problem object is returned.

```python
        problem = MagneticProblemAxiStatic("Problem conductor loop", mesh, shape_function, regions, 
                                           materials, boundary_cond, excitations)
        return problem
```

### Summary
The complete code to create the torus conductio with all necessary imports reads:

In [ ]:
import gmsh
import numpy as np
from dataclasses import dataclass

from scipy.constants import mu_0
from pyrit.bdrycond import BCDirichlet, BdryCond
from pyrit.geometry import Geometry
from pyrit.material import Mat, Materials, Reluctivity, Conductivity
from pyrit.excitation import Excitations, CurrentDensity
from pyrit.mesh import AxiMesh
from pyrit.shapefunction import TriAxisymmetricEdgeShapeFunction

from pyrit.problem import MagneticProblemAxiStatic

nu_0 = 1 / mu_0

In [ ]:
@dataclass
class ConductorLoop:
    width: float = 2.
    height: float = 4.
    center_conductor_x: float = 1.
    center_conductor_y: float = 2.
    radius_conductor: float = 0.2

    current: float = 1.
    conductivity: float = 1e6

    @property
    def current_density(self) -> float:
        """The current density in the conductor"""
        return self.current / (np.pi * self.radius_conductor ** 2)

    def create_problem(self, refinement_steps: int = 0, **kwargs) -> MagneticProblemAxiStatic:
        """Create the problem.

        Parameters
        ----------
        refinement_steps : int
            Number of refinement steps in the meshing process.
        kwargs :
            Keyword arguments passed to the constructor of ``Geometry``

        Returns
        -------
        problem : MagneticProblemAxiStatic
            A problem instance
        """
        geo = Geometry(model_name="Test MagneticProblem", **kwargs)

        # Materials
        air = Mat('air', Reluctivity(nu_0), Conductivity(0))
        copper = Mat('copper', Reluctivity(nu_0), Conductivity(self.conductivity))
        materials = Materials(air, copper)

        # Boundary conditions
        outer_bc = BCDirichlet(0)
        boundary_cond = BdryCond(outer_bc)

        # Excitation
        exci = CurrentDensity(self.current_density, 'cd')
        excitations = Excitations(exci)

        # Creating physical groups
        pg_conductor = geo.create_physical_group(tag=1, dim=2, name="conductor")
        pg_surrounding = geo.create_physical_group(tag=2, dim=2, name="surrounding")
        pg_boundary = geo.create_physical_group(tag=3, dim=1, name="boundary")

        # Assignment of physical groups to materials, boundary conditions and excitations
        geo.add_material_to_physical_group(air, pg_surrounding)
        geo.add_material_to_physical_group(copper, pg_conductor)
        geo.add_boundary_condition_to_physical_group(outer_bc, pg_boundary)
        geo.add_excitation_to_physical_group(exci, pg_conductor)

        # Build geometry
        with geo:
            rectangle = gmsh.model.occ.addRectangle(0, 0, 0, self.width, self.height)
            disk = gmsh.model.occ.addDisk(self.center_conductor_x, self.center_conductor_y, 0, self.radius_conductor,
                                          self.radius_conductor)
            gmsh.model.occ.cut([[2, rectangle]], [[2, disk]], removeTool=False)

            pg_conductor.add_entity(disk)
            pg_boundary.add_entity(6, 8, 9)
            pg_surrounding.add_entity(rectangle)

            # Create mesh
            geo.create_mesh(dim=2)
            for _ in range(refinement_steps):
                geo.refine_mesh()
            mesh = geo.get_mesh(dim=2)
            mesh = AxiMesh.generate_axi_mesh(mesh)
            
            # Create regions
            regions = geo.get_regions()

        # Create shape function object
        shape_function = TriAxisymmetricEdgeShapeFunction(mesh)

        problem = MagneticProblemAxiStatic("Problem conductor loop", mesh, shape_function, 
                                           regions, materials, boundary_cond, excitations)
        return problem

### Execution of code
Copy the given code given for example in a separate python file to define the model. Create the geometry with the following commands. Because we used a dataclass for ``ConductorLoop``, it is very simple to modify the the geometry or the add new variables to the geometry. The mesh size can also easily be set using the ``kwargs`` of the ``create_problem``-method. 

Exemplary, the current through the conductor and the conductors radius are customized here, as well as the mesh. Furthermore, the GUI of gmsh can be set to open after geometry creation.

In [ ]:
conductor_loop = ConductorLoop(current=4, radius_conductor=0.1)
problem = conductor_loop.create_problem(mesh_size_factor = 0.3, refinement_steps=2, show_gui=True)
print(problem)

## Approach 3: Model definition based on geo-file
The third approach uses an existing *Gmsh* file (.geo file) with already defined physical groups. 

This approach is demonstrated by modelling a simple cylindrical resistor. The cylindrical resistor consists of two layered materials, which are located between two electrodes to which a voltage of $V =1 $V is applied. The conductivity of the inner material is 2 S/m. Due to the axial symmetry of this configuration, it is sufficient to model a cut of the model in the $\rho-z$ plane.

In the following, it is assumed that the model is given as a .geo file. The following command opens the file in the *Gmsh* GUI:

In [ ]:
from pyrit.geometry import Geometry
import gmsh
geo = Geometry("",show_gui=True)
with geo:
    gmsh.merge('Tutorial_GEO.geo')

With ``Tools->Visibility or [Strg+Shift+V]`` we can take a look at the physical groups of the model. We can see that the model has four physical groups: The physical groups INNER_MATERIAL and OUTER_MATERIAL are allocated on surfaces and represent the two materials of the model. The physical groups GROUND and VOLTAGE are located on curves and indicate the position of the boundary conditions that are needed to apply the excitation voltage. 

Similar to approach 2, the model is represented in *Pyrit* by a dataclass that handles the properties of the model, e.g. the applied voltage and material parameters. A dedicated method ``create_problem`` of the class creates an instance of a specific problem class, in which all data structures needed to define the model are combined. 

In the following, the code is first shown and explained in smaller parts. Then, the code is summarized and executed for a simple example.

### Head of the class
The model is implemented as a Python dataclass in Python. The head of the class looks for example like in the following cell. 
Basic properties are provided firstly, i.e. the conductivities of the materials and the potentials at the electrodes. Secondly, the path to the .geo file is provided. 

```python
@dataclass
class CylindricalResistor:
    # Model parameters
    conductivity_inner: float = 2  # [S/m]
    conductivity_outer: float = 1  # [S/m]
    ground_potential: float = 0  # [V]
    excitation_voltage: float = 1  # [V]

    # Parameters for geometry import from gmsh
    path_to_geo_file: Union[str, 'Path'] = 'Tutorial_GEO'
```

### Method create_problem
The dataclass contains a method ``create_problem`` to create the problem. Inside of this method, the materials and boundary conditions are defined and connected to the geometry of the model. Subsequently, all data structures needed to define the problem are combined in an instance of a specific problem class. The argument ``refinement_steps`` provides the number of refinement steps during the meshing process, ``show_gui`` determines whether the mesh is opened in the *Gmsh* GUI. 

```python
    def create_problem(self, refinement_steps: int = 0, show_gui: bool = False) -> CurrentFlowProblemAxiStatic:
        """Create the problem.

        Parameters
        ----------
        refinement_steps : int
            Number of refinement steps in the meshing process.
        show_gui: bool
            Open the GMSH GUI?

        Returns
        -------
        problem : CurrentFlowProblemAxiStatic
            A problem instance
        """
```

#### Materials
First, we define the materials. Each is an instance of ``Mat`` and holds a number of material properties. In our example, we define the electric conductivity as it is the relevant property for a stationary current simulation. In order for *Pyrit* to later link the materials to the mesh, it is essential that the names of the materials match the names of the physical groups defined in the .geo file. Therefore, in this example, the materials must be called INNER_MATERIAL and OUTER_MATERIAL, respectively.

The two materials are added to a ``Materials`` object, which manages all materials of a problem.

```python
        inner_material = Mat('INNER_MATERIAL', Conductivity(self.conductivity_inner))
        outer_material = Mat('OUTER_MATERIAL', Conductivity(self.conductivity_outer))
        materials = Materials(inner_material, outer_material)
```

#### Boundary conditions
Next, the boundary conditions are defined. In order to apply the voltage $V$ to the model, two Dirichlet boundary conditions are defined. The first defines the ground potential at the inner electrode, the second applies the excitation voltage to the outer electrode. Similar to the material definition, it is crucial that the names of the boundary conditions match the names of the physical groups in the .geo file.

The boundary conditions are added to a ``BdryCond`` object. This is responsible for managing all the different boundary conditions present in a problem.

```python
        bc_ground = BCDirichlet(self.ground_potential, name="GROUND")
        bc_voltage = BCDirichlet(self.excitation_voltage, name="VOLTAGE")
        boundary_cond = BdryCond(bc_ground, bc_voltage)
```

#### Mesh generation
For the mesh generation, the method ``geo_to_msh`` creates a .msh file and returns its file path. Its input parameters are the path to the .geo file and the number of refinement steps. The .msh file is then read by the  method ``read_msh_file`` and a ``Mesh`` object is generated. Since the problem is axisymmetric, the output type is set to ``AxiMesh``.

```python
        path_to_msh_file = geo_to_msh(self.path_to_geo_file, refinement_steps=refinement_steps)
        mesh = read_msh_file(path_to_msh_file, mesh_type=AxiMesh)
```

#### Regions generation
In the next step, the material properties and boundary conditions are assigned to the geometry. A region maps the physical properties to corresponding mesh entities (such as triangles or edges). This is done by the method ``generate_regions``, which automatically connects materials and boundary conditions to mesh entities belonging to physical groups with the same name. This is the reason why it is crucial to choose the names of the materials and boundary conditions as specified in the .geo file.
 
```python
        regions = generate_regions(self.path_to_msh_file, materials, boundary_cond)
```

#### Creation of the shape function object
With the mesh, we create the shape function object that will later create the finite element matrices. 

```python
        shape_function = TriAxisymmetricNodalShapeFunction(mesh)
```

#### Creation of the Problem
We now have a lot of different objects that contain different information about our problem. The last step of this method is to combine them in one problem-class. So all data structures are tied together and further handling is simplified. 

In the next step, we initialize the stationary current problem in axisymmetric coordinates with the template-class ``CurrentFlowProblemAxiStatic``. The constructor expects all of the created data structures. The problem object itself performs consistency and type checks to prevent faulty inputs. It also provides a ``solve``-method to solve the problem in a standardized way. Finally, the problem object is returned.

```python
        problem = CurrentFlowProblemAxiStatic("Cylindrical Resistor", mesh, shape_function, regions, 
                                           materials, boundary_cond, excitations)
        return problem
```

### Summary
The complete code to create the cylindrical resistor with all necessary imports reads:

In [1]:
from dataclasses import dataclass
from typing import Union
from pathlib import Path

from pyrit.material import Mat, Materials, Conductivity
from pyrit.bdrycond import BdryCond, BCDirichlet
from pyrit.toolbox.ImportGmshToolbox import geo_to_msh, read_msh_file, generate_regions
from pyrit.mesh import AxiMesh
from pyrit.shapefunction import TriAxisymmetricNodalShapeFunction
from pyrit.problem import CurrentFlowProblemAxiStatic


In [2]:
@dataclass
class CylindricalResistor:
    # Model parameters
    conductivity_inner: float = 1  # [S/m]
    conductivity_outer: float = 2  # [S/m]
    ground_potential: float = 0  # [V]
    excitation_voltage: float = 1  # [V]

    # Parameters for geometry import from gmsh
    path_to_geo_file: Union[str, 'Path'] = 'Tutorial_GEO'
        
    def create_problem(self, refinement_steps: int = 0, show_gui: bool = False) -> CurrentFlowProblemAxiStatic:
        """Create the problem.

        Parameters
        ----------
        refinement_steps : int
            Number of refinement steps in the meshing process.
        show_gui: bool
            Open the GMSH GUI?

        Returns
        -------
        problem : CurrentFlowProblemAxiStatic
            A problem instance
        """
        
        # Materials
        inner_material = Mat('INNER_MATERIAL', Conductivity(self.conductivity_inner))
        outer_material = Mat('OUTER_MATERIAL', Conductivity(self.conductivity_outer))
        materials = Materials(inner_material, outer_material)
        
        # Boundary conditions
        bc_ground = BCDirichlet(self.ground_potential, name="GROUND")
        bc_voltage = BCDirichlet(self.excitation_voltage, name="VOLTAGE")
        boundary_cond = BdryCond(bc_ground, bc_voltage)
        
        # Mesh 
        path_to_msh_file = geo_to_msh(self.path_to_geo_file, refinement_steps=refinement_steps, show_gui=show_gui)
        mesh = read_msh_file(path_to_msh_file, mesh_type=AxiMesh)
        
        # Regions
        regions = generate_regions(path_to_msh_file, materials, boundary_cond)
        
        # Shape functions
        shape_function = TriAxisymmetricNodalShapeFunction(mesh)
        
        # Problem 
        problem = CurrentFlowProblemAxiStatic("Cylindrical Resistor", mesh, shape_function, regions, 
                                           materials, boundary_cond)
        return problem

### Execution of code
To create the model, we first create an instance of the CylindricalResistor class. Since CylindricalResistor is implemented as a dataclass, it is very simple to modify the model paramters. The parameters can either be set directly during initialization or later using the dot notation. Exemplarily, the conductivities of the materials are customized here. Once all parameters are set to the desired value, the problem can be created using the ``create_problem``-method. 


In [3]:
cylindrical_resistor = CylindricalResistor(conductivity_inner = 4)
cylindrical_resistor.conductivity_outer = 3
problem = cylindrical_resistor.create_problem(refinement_steps=1, show_gui=True)
print(problem)

AttributeError: 'CylindricalResistor' object has no attribute 'path_to_msh_file'